# 01_Análisis exploratorio de datos (TCGA_BRCA)

## Análisis Exploratorio Inicial de Datos de Expresión y Metadatos Clínicos (TCGA-BRCA)

**Autor:** Javier García Delgado  
**Fecha:** 24 de mayo de 2025  
**Proyecto:** Análisis RNA-Seq de cáncer de mama (TCGA-BRCA)  
**Notebook 1 de 5  


En este notebook se realiza la carga, revisión y exploración inicial de los datos de expresión génica (RNA-Seq) y los metadatos clínicos asociados al estudio TCGA-BRCA.  
El objetivo es comprender la estructura y calidad de los datos antes de proceder con análisis específicos como expresión diferencial, clustering o modelos supervisados.


## 1. Importar paquetes

In [2]:
import os
import numpy as np
import pandas as pd

# Autocompletar rápido
%config IPCompleter.greedy=True

base_path = "D:proyectis_genomica/rna_seq_tcga_brca"

folders = [
    "data",
    "notebooks",
    "results",
    "src"
]

for folder in folders:
    os.makedirs(os.path.join(base_path, folder), exist_ok=True)

Al ser .tsv los archivos indicamos sep='\t' para que los lea como .csv

## 2. Carga de datos
   
Cargamos la matriz de expresión génica y los metadatos clínicos desde UCSC Xena.

In [3]:
# Cargar datos de expresión y fenotipo
expresion = pd.read_csv("D:/proyectos_genomica/rna_seq_tcga_brca/data/tcga_brca_rnaseq.tsv", sep='\t', index_col=0)
metadatos = pd.read_csv("D:/proyectos_genomica/rna_seq_tcga_brca/data/tcga_brca_phenotype.tsv", sep='\t', index_col=0)

# Mostrar dimensiones
print("Expresión:", expresion.shape)
print("Metadatos:", metadatos.shape)

Expresión: (20530, 1218)
Metadatos: (1247, 193)


Observamos que la matriz expresión contiene 20.530 genes (filas) y 1218 muestas (columnas)

El archivo de metadatos contiene información clínica para 1247 muestras y 193 variables.

## 3. Información general de los datos

Consultamos la columna 'sample_type' de los metadatos para comprobar si el dataset incluye tanto muestras tumorales como normales. Esto es imprescindible para realizar un análisis de expresión diferencial entre grupos.

In [4]:
# Ver los tipos de muestra disponibles
metadatos['sample_type'].value_counts()

sample_type
Primary Tumor          1101
Solid Tissue Normal     139
Metastatic                7
Name: count, dtype: int64

Confirmamos que tenemos 1101 muestras tumorales (Primary Tumor) y 139 normales (Solid Tissue Normal).

En lugar de combinar los datos clínicos y de expresión en un único DataFrame, mantendremos por separado debido al elevado número de genes y a la estructura de los datos transcriptómicos.

Al identificar genes relevantes se construirá un DF combinado para análisis clínico-molecular integrado.

### Estructura general

Revisamos la estructura general del DataFrame de expresión y de metadatos con 'info()'

In [5]:
expresion.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20530 entries, ARHGEF10L to SELS
Columns: 1218 entries, TCGA-AR-A5QQ-01 to TCGA-B6-A0X1-01
dtypes: float64(1218)
memory usage: 190.9+ MB


In [6]:
metadatos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1247 entries, TCGA-3C-AAAU-01 to TCGA-Z7-A8R6-01
Columns: 193 entries, AJCC_Stage_nature2012 to _GENOMIC_ID_TCGA_BRCA_PDMarray
dtypes: float64(34), int64(1), object(158)
memory usage: 1.8+ MB


Respecto a los tipos de datos: 
- En 'expresion', todas las columnas son numéricas ('float64'), lo que indica niveles de expresión génica.
- En 'metadatos', predominan las variables categóricas ('object'), pero también hay 34 numéricas('float64') y 1 discreta('int64').

Esto es útil para decidir qué columnas pueden analizarse cuantitativamente y cuáles requieren codificación previa.

### Detección de valores ausentes

In [11]:
# Total de nulos en 'expresion'
expresion.isna().sum().sum()      

np.int64(0)

In [10]:
# Las 10 columnas con más nulos en 'metadatos'
metadatos.isna().sum().sort_values(ascending=False).head(10)  

mtsttc_brst_crcnm_hr2_rbb_ps_fndng_flrscnc_n_st_hybrdztn_clcltn     1246
hr2_n_nd_cntrmr_17_cpy_nmbr_mtsttc_brst_crcnm_nlyss_npt_ttl_nmbr    1245
metastatic_breast_carcinoma_pos_finding_hr2_rbb2_thr_msr_scl_txt    1245
days_to_last_known_alive                                            1245
days_to_additional_surgery_locoregional_procedure                   1245
her2_neu_metastatic_breast_carcinoma_copy_analysis_inpt_ttl_nmbr    1244
mtsttc_brst_crcnm_flrscnc_n_st_hybrdztn_dgnstc_prc_cntrmr_17_sgn    1244
metastatic_breast_carcinoma_her2_erbb_method_calculatin_mthd_txt    1244
metastatic_breast_carcinm_ps_fndng_prgstrn_rcptr_thr_msr_scl_txt    1243
pos_finding_metastatic_brst_crcnm_strgn_rcptr_thr_msrmnt_scl_txt    1243
dtype: int64

Comprobamos que hay columnas con casi todos los valores nulos que aparecen.

Son campos clínicos muy específicos que no deberían ser relevantes para el análisis, ya que muchas de las cuales solo aplican a ciertos subtipos tumorales. Se revisarán más adelante si se utilizarán en análisis clínico-molecular específico.

Por tanto estas columnas no las usaremos en el análisis principal y no es necesario imputar ni anular todavía para evitar sesgos.

### Estadísticos descriptivos

Pasamos un .describe() para ver estadísticas numéricas

In [12]:
expresion.describe()

,TCGA-AR-A5QQ-01,TCGA-D8-A1JA-01,TCGA-BH-A0BQ-01,TCGA-BH-A0BT-01,TCGA-A8-A06X-01,TCGA-A8-A096-01,TCGA-BH-A0C7-01,TCGA-AC-A5XU-01,TCGA-PE-A5DE-01,TCGA-PE-A5DC-01,...,TCGA-A7-A13E-11,TCGA-C8-A8HP-01,TCGA-E9-A5FL-01,TCGA-AC-A2FB-11,TCGA-E2-A15F-01,TCGA-A2-A3XT-01,TCGA-B6-A0X7-01,TCGA-BH-A1EV-11,TCGA-3C-AALJ-01,TCGA-B6-A0X1-01
count,20530.000000,20530.000000,20530.000000,20530.000000,20530.000000,20530.000000,20530.000000,20530.000000,20530.000000,20530.000000,...,20530.000000,20530.000000,20530.000000,20530.000000,20530.000000,20530.000000,20530.000000,20530.000000,20530.00000,20530.000000
mean,6.458289,6.196788,6.532558,6.396285,6.195509,6.402330,6.471376,6.450572,6.465499,6.538643,...,6.529149,6.580747,6.636886,6.654616,6.459149,6.409939,6.495686,6.692792,6.44033,6.396076
std,4.080824,4.156713,4.024625,4.064396,4.159198,4.129318,4.040228,4.058813,4.039765,4.001138,...,4.023284,3.987771,3.954669,3.972979,4.027358,4.071828,4.014227,3.934542,4.05272,4.029709
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,2.577200,1.890900,2.913300,2.485900,1.800700,2.119000,2.634600,2.598000,2.662200,2.914625,...,2.880125,3.051600,3.418200,3.280900,2.698800,2.452500,2.911200,3.405800,2.68700,2.558800
50%,7.601700,7.188900,7.701800,7.477100,7.365700,7.706900,7.681100,7.685800,7.709500,7.699700,...,7.666550,7.720050,7.643600,7.838700,7.619300,7.584100,7.681300,7.818300,7.62080,7.486100
75%,9.699700,9.622175,9.694100,9.705550,9.635100,9.719300,9.716600,9.691400,9.691000,9.727000,...,9.695800,9.716600,9.698625,9.750550,9.701300,9.712075,9.715625,9.752925,9.68330,9.694900
max,18.706900,17.372800,18.057500,17.242500,17.062800,17.771600,17.260700,17.480100,17.218000,16.607100,...,17.842700,17.522600,17.753600,17.305200,17.706300,16.528900,18.651600,16.884100,17.46420,17.042800


In [13]:
metadatos.describe()

,Age_at_Initial_Pathologic_Diagnosis_nature2012,CN_Clusters_nature2012,Days_to_Date_of_Last_Contact_nature2012,Days_to_date_of_Death_nature2012,Integrated_Clusters_no_exp__nature2012,Integrated_Clusters_unsup_exp__nature2012,Integrated_Clusters_with_PAM50__nature2012,OS_Time_nature2012,OS_event_nature2012,SigClust_Intrinsic_mRNA_nature2012,...,initial_weight,lymph_node_examined_count,metastatic_breast_carcinoma_her2_neu_chromosone_17_signal_rat_vl,methylation_Clusters_nature2012,miRNA_Clusters_nature2012,mtsttc_brst_crcnm_flrscnc_n_st_hybrdztn_dgnstc_prc_cntrmr_17_sgn,number_of_lymphnodes_positive_by_he,number_of_lymphnodes_positive_by_ihc,sample_type_id,year_of_initial_pathologic_diagnosis
count,952.000000,773.000000,883.000000,135.00000,348.000000,348.000000,348.000000,952.000000,952.000000,522.000000,...,1051.000000,1110.000000,9.000000,799.000000,694.000000,3.000000,1068.000000,376.000000,1247.000000,1240.000000
mean,57.720588,2.712807,835.907135,1819.02963,2.195402,2.640805,2.787356,936.422269,0.141807,-6.764368,...,319.971456,10.696396,1.645556,3.111389,4.175793,2.566667,2.360487,0.271277,2.142743,2007.634677
std,13.430495,1.332264,989.399560,1114.39515,1.011052,0.995813,0.911446,1060.241798,0.349035,4.813213,...,272.945310,8.765798,1.039881,1.387866,1.662138,0.472582,4.645665,0.763029,3.160576,4.408290
min,26.000000,1.000000,0.000000,157.00000,1.000000,1.000000,1.000000,0.000000,0.000000,-13.000000,...,10.000000,0.000000,0.950000,1.000000,1.000000,2.200000,0.000000,0.000000,1.000000,1988.000000
25%,48.000000,2.000000,170.500000,921.00000,1.000000,2.000000,2.000000,194.500000,0.000000,-12.000000,...,140.000000,3.000000,1.000000,2.000000,3.000000,2.300000,0.000000,0.000000,1.000000,2006.000000
50%,58.000000,3.000000,517.000000,1692.00000,2.000000,3.000000,3.000000,576.000000,0.000000,-5.000000,...,230.000000,9.000000,1.200000,3.000000,4.000000,2.400000,1.000000,0.000000,1.000000,2009.000000
75%,66.250000,4.000000,1213.500000,2534.00000,3.000000,3.000000,3.000000,1321.000000,0.000000,-2.000000,...,400.000000,16.000000,1.700000,4.000000,6.000000,2.750000,2.000000,0.000000,1.000000,2010.000000
max,90.000000,5.000000,7125.000000,4456.00000,5.000000,5.000000,4.000000,7125.000000,1.000000,0.000000,...,2190.000000,44.000000,4.100000,5.000000,7.000000,3.100000,35.000000,8.000000,11.000000,2013.000000


### Tipos de datos

In [7]:
expresion.dtypes.value_counts()

float64    1218
Name: count, dtype: int64

In [8]:
metadatos.dtypes.value_counts()

object     158
float64     34
int64        1
Name: count, dtype: int64

### Visualizar estructura

Mostramos las primeras filas de los DFs con .head() para verificar la estructura inicial antes del filtrado.

In [9]:
expresion.head()

,TCGA-AR-A5QQ-01,TCGA-D8-A1JA-01,TCGA-BH-A0BQ-01,TCGA-BH-A0BT-01,TCGA-A8-A06X-01,TCGA-A8-A096-01,TCGA-BH-A0C7-01,TCGA-AC-A5XU-01,TCGA-PE-A5DE-01,TCGA-PE-A5DC-01,...,TCGA-A7-A13E-11,TCGA-C8-A8HP-01,TCGA-E9-A5FL-01,TCGA-AC-A2FB-11,TCGA-E2-A15F-01,TCGA-A2-A3XT-01,TCGA-B6-A0X7-01,TCGA-BH-A1EV-11,TCGA-3C-AALJ-01,TCGA-B6-A0X1-01
sample,,,,,,,,,,,,,,,,,,,,,
ARHGEF10L,9.5074,7.4346,9.3216,9.0198,9.6417,9.7665,10.0931,9.1524,9.9398,9.6287,...,9.6265,10.1826,9.9199,9.9090,10.0334,11.5144,10.5745,9.4048,10.9468,10.3164
HIF3A,1.5787,3.6607,2.7224,1.3414,0.5819,0.2738,3.6090,0.4738,2.9378,4.1136,...,8.1546,2.2159,3.8645,8.1872,0.8836,1.3169,4.0696,7.2537,0.9310,2.4191
RNF17,0.0000,0.6245,0.5526,0.0000,0.0000,0.8765,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,3.7305,0.0000,0.0000,1.1329,0.4258,0.0000,0.0000,0.0000
RNF10,11.3676,11.9181,11.9665,13.1881,12.0036,11.8118,11.3820,11.5004,12.2055,12.1312,...,11.9869,12.2653,12.4815,11.8263,12.0135,11.5818,11.8663,11.5460,12.2616,12.1570
RNF11,11.1292,13.5273,11.4105,11.0911,11.2545,10.8554,10.7663,10.4358,11.2210,10.8013,...,11.9344,11.4117,10.4902,11.5754,10.8370,10.2736,10.7644,11.8774,10.7842,11.2420


Observamos que cada columna representa una muestra y cada fila es un gen.

In [16]:
metadatos.head()

,AJCC_Stage_nature2012,Age_at_Initial_Pathologic_Diagnosis_nature2012,CN_Clusters_nature2012,Converted_Stage_nature2012,Days_to_Date_of_Last_Contact_nature2012,Days_to_date_of_Death_nature2012,ER_Status_nature2012,Gender_nature2012,HER2_Final_Status_nature2012,Integrated_Clusters_no_exp__nature2012,...,_GENOMIC_ID_TCGA_BRCA_mutation_wustl_gene,_GENOMIC_ID_TCGA_BRCA_miRNA_GA,_GENOMIC_ID_TCGA_BRCA_exp_HiSeqV2_percentile,_GENOMIC_ID_data/public/TCGA/BRCA/miRNA_GA_gene,_GENOMIC_ID_TCGA_BRCA_gistic2thd,_GENOMIC_ID_data/public/TCGA/BRCA/miRNA_HiSeq_gene,_GENOMIC_ID_TCGA_BRCA_G4502A_07_3,_GENOMIC_ID_TCGA_BRCA_exp_HiSeqV2,_GENOMIC_ID_TCGA_BRCA_gistic2,_GENOMIC_ID_TCGA_BRCA_PDMarray
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-3C-AAAU-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6ef883fc-81f3-4089-95e0-86904ffc0d38,NaN,TCGA-3C-AAAU-01A-11D-A41E-01,TCGA-3C-AAAU-01,NaN,6ef883fc-81f3-4089-95e0-86904ffc0d38,TCGA-3C-AAAU-01A-11D-A41E-01,NaN
TCGA-3C-AALI-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,dd8d3665-ec9d-45be-b7b9-a85dac3585e2,NaN,TCGA-3C-AALI-01A-11D-A41E-01,TCGA-3C-AALI-01,NaN,dd8d3665-ec9d-45be-b7b9-a85dac3585e2,TCGA-3C-AALI-01A-11D-A41E-01,NaN
TCGA-3C-AALJ-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,c924c2a8-ab41-4499-bb30-79705cc17d45,NaN,TCGA-3C-AALJ-01A-31D-A41E-01,TCGA-3C-AALJ-01,NaN,c924c2a8-ab41-4499-bb30-79705cc17d45,TCGA-3C-AALJ-01A-31D-A41E-01,NaN
TCGA-3C-AALK-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1a19e068-d213-41ef-aebb-104017a883cc,NaN,TCGA-3C-AALK-01A-11D-A41E-01,TCGA-3C-AALK-01,NaN,1a19e068-d213-41ef-aebb-104017a883cc,TCGA-3C-AALK-01A-11D-A41E-01,NaN
TCGA-4H-AAAK-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2ea9e472-a408-4ae0-975d-50a566f22b2a,NaN,TCGA-4H-AAAK-01A-12D-A41E-01,TCGA-4H-AAAK-01,NaN,2ea9e472-a408-4ae0-975d-50a566f22b2a,TCGA-4H-AAAK-01A-12D-A41E-01,NaN


Confirmamos que en metadatos los IDs están en el índice (sampleID) y las variables clínicas como columnas.

## 4. Revisión de IDs

Comprobamos si los identificadores de muestra de los archivos de expresión y metadatos son compatibles debido a su formato.

Esto es fundamental para poder alinear ambos datasets sin pérdidas antes de cualquier análisis conjunto.

Verificamos que ambos están en formato 'TCGA-XX-YYYY-01', por lo que no es necesario recortar ni transformar los IDs.

Calculamos directamente ls muestras comunes.

In [17]:
# Calcular la intersección de IDs
muestras_comunes = expresion.columns.intersection(metadatos.index)
print("Número de muestras comunes:", len(muestras_comunes))

# Mostrar algunos ejemplos de IDs comunes
muestras_comunes[:5].tolist()

Número de muestras comunes: 1218


['TCGA-AR-A5QQ-01',
 'TCGA-D8-A1JA-01',
 'TCGA-BH-A0BQ-01',
 'TCGA-BH-A0BT-01',
 'TCGA-A8-A06X-01']

## 5. Filtrado de datos

Filtramos ambos DataFrames ('expresion' y 'metadatos') para conservar únicamente las muestras comunes.

Además, transponemos la matriz de 'expresion' para que cada fila represente una muestra ( como en 'metadatos') y cada columna represente un gen, facilitando los análisis posteriores por muestra.

In [18]:
# Aplicamos el filtrado a ambos DataFrames
exp_filtrado = expresion[muestras_comunes].T  # Transpuesta para tener muestras en filas
meta_filtrado = metadatos.loc[muestras_comunes]

# Verificar dimensiones
print("Dimensiones de expresión filtrada:", exp_filtrado.shape)
print("Dimensiones de metadatos filtrados:", meta_filtrado.shape)


Dimensiones de expresión filtrada: (1218, 20530)
Dimensiones de metadatos filtrados: (1218, 193)


Mostramos las primeras filas de los DataFrames filtrados (`exp_filtrado` y `meta_filtrado`) para verificar que la orientación y estructura son correctas antes de continuar.

In [20]:
# Vista preliminar tras el filtrado
print("exp_filtrado:")
display(exp_filtrado.head())


exp_filtrado:


sample,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,REM1,MTVR2,RTN4RL2,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
TCGA-AR-A5QQ-01,9.5074,1.5787,0.0000,11.3676,11.1292,9.9722,11.5966,3.2396,0.0000,3.5764,...,0.5819,0.0000,5.3307,0.5819,9.1928,13.8808,7.3830,11.4289,7.8456,10.7384
TCGA-D8-A1JA-01,7.4346,3.6607,0.6245,11.9181,13.5273,10.8702,12.3048,2.5547,0.0000,6.0854,...,1.0589,0.0000,3.1017,3.7793,9.8340,10.7066,7.2467,9.1673,0.0000,10.1225
TCGA-BH-A0BQ-01,9.3216,2.7224,0.5526,11.9665,11.4105,10.4406,12.8186,4.7115,0.0000,5.8329,...,0.0000,2.7224,4.2976,0.0000,9.2304,12.9973,8.3374,11.3400,8.3765,9.9367
TCGA-BH-A0BT-01,9.0198,1.3414,0.0000,13.1881,11.0911,10.4244,12.6427,2.7553,0.0000,4.6308,...,0.0000,2.9823,4.3067,0.0000,8.7816,12.3298,7.7036,11.3025,7.0585,9.3784
TCGA-A8-A06X-01,9.6417,0.5819,0.0000,12.0036,11.2545,10.1480,12.6622,4.2765,1.8007,4.4505,...,1.3163,0.5819,3.5764,0.0000,9.3024,11.3338,7.7186,10.6898,7.0992,11.1740


In [21]:
# Vista preliminar tras el filtrado 
print("\nmeta_filtrado:")
display(meta_filtrado.head())


meta_filtrado:


,AJCC_Stage_nature2012,Age_at_Initial_Pathologic_Diagnosis_nature2012,CN_Clusters_nature2012,Converted_Stage_nature2012,Days_to_Date_of_Last_Contact_nature2012,Days_to_date_of_Death_nature2012,ER_Status_nature2012,Gender_nature2012,HER2_Final_Status_nature2012,Integrated_Clusters_no_exp__nature2012,...,_GENOMIC_ID_TCGA_BRCA_mutation_wustl_gene,_GENOMIC_ID_TCGA_BRCA_miRNA_GA,_GENOMIC_ID_TCGA_BRCA_exp_HiSeqV2_percentile,_GENOMIC_ID_data/public/TCGA/BRCA/miRNA_GA_gene,_GENOMIC_ID_TCGA_BRCA_gistic2thd,_GENOMIC_ID_data/public/TCGA/BRCA/miRNA_HiSeq_gene,_GENOMIC_ID_TCGA_BRCA_G4502A_07_3,_GENOMIC_ID_TCGA_BRCA_exp_HiSeqV2,_GENOMIC_ID_TCGA_BRCA_gistic2,_GENOMIC_ID_TCGA_BRCA_PDMarray
TCGA-AR-A5QQ-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,16f90247-c9a5-40d5-a3a1-08bba2d1d266,NaN,TCGA-AR-A5QQ-01A-11D-A28A-01,TCGA-AR-A5QQ-01,NaN,16f90247-c9a5-40d5-a3a1-08bba2d1d266,TCGA-AR-A5QQ-01A-11D-A28A-01,NaN
TCGA-D8-A1JA-01,Stage X,60.0,1.0,Stage IIIB,18.0,NaN,Negative,FEMALE,Positive,NaN,...,NaN,NaN,ae81378c-ada3-43cb-9855-941467fd856c,NaN,TCGA-D8-A1JA-01A-11D-A13J-01,TCGA-D8-A1JA-01,NaN,ae81378c-ada3-43cb-9855-941467fd856c,TCGA-D8-A1JA-01A-11D-A13J-01,NaN
TCGA-BH-A0BQ-01,Stage I,39.0,2.0,Stage I,826.0,NaN,Positive,FEMALE,Negative,3.0,...,TCGA-BH-A0BQ-01A-21D-A10Y-09,NaN,2ada4fe0-57f2-4a54-9580-4e0ae45a2f65,NaN,TCGA-BH-A0BQ-01A-21D-A111-01,TCGA-BH-A0BQ-01,TCGA-BH-A0BQ-01A-21R-A115-07,2ada4fe0-57f2-4a54-9580-4e0ae45a2f65,TCGA-BH-A0BQ-01A-21D-A111-01,TCGA-BH-A0BQ-01
TCGA-BH-A0BT-01,Stage IIA,56.0,4.0,Stage IIA,1386.0,NaN,Positive,FEMALE,Negative,3.0,...,TCGA-BH-A0BT-01A-11D-A12Q-09,NaN,fe8764cf-04ee-480d-96ce-09d0d388f4f6,NaN,TCGA-BH-A0BT-01A-11D-A12N-01,TCGA-BH-A0BT-01,TCGA-BH-A0BT-01A-11R-A12P-07,fe8764cf-04ee-480d-96ce-09d0d388f4f6,TCGA-BH-A0BT-01A-11D-A12N-01,TCGA-BH-A0BT-01
TCGA-A8-A06X-01,Stage IIB,77.0,1.0,Stage IIB,943.0,943.0,Positive,FEMALE,Positive,NaN,...,TCGA-A8-A06X-01A-21W-A019-09,NaN,3ea9e7f1-39af-4089-b1c0-3c62db4e3102,NaN,TCGA-A8-A06X-01A-21D-A011-01,NaN,TCGA-A8-A06X-01A-21R-A00Z-07,3ea9e7f1-39af-4089-b1c0-3c62db4e3102,TCGA-A8-A06X-01A-21D-A011-01,TCGA-A8-A06X-01


Observamos que `exp_filtrado` contiene 20530 columnas, correspondientes a genes, y 1218 filas, correspondientes a muestras.  

Esta estructura confirma que la transposición se ha realizado correctamente y que estamos listos para comenzar el análisis por muestra.


- Por el momento, no combinamos la matriz de expresión y los metadatos en un único DataFrame.
- Es preferible mantenerlos separados, ya que los análisis transcriptómicos suelen requerir estructuras diferenciadas:  
    - una matriz numérica de alta dimensión (`exp_filtrado`)
    -  una tabla clínica (`meta_filtrado`) que define los grupos y variables asociadas.
- Se integrarán puntualmente más adelante si es necesario para análisis específicos.


## Conclusiones 

En esta primera fase del análisis:

- Hemos cargado correctamente la matriz de expresión génica (20530 genes × 1218 muestras) y los metadatos clínicos (193 variables × 1247 muestras).
- Verificamos la estructura, tipos de datos y valores nulos, manteniendo separados los datos transcriptómicos y clínicos.
- Identificamos 1218 muestras comunes entre ambos archivos, que han sido correctamente alineadas.
- Confirmamos que `exp_filtrado` está orientado por muestras (filas) y genes (columnas), y `meta_filtrado` por muestras y variables clínicas.

Estos DataFrames están listos para comenzar el preprocesado y análisis exploratorio por grupo en el siguiente notebook.
